# Main Dashboard

In [1]:
# import libraries
from sklearn import metrics
from sklearn.model_selection import train_test_split
import lightgbm as ltb
import xgboost as xgb
import catboost as catb
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from explainerdashboard import ClassifierExplainer, ExplainerDashboard, ExplainerHub
from explainerdashboard.custom import *
import re


The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`

The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`


In [4]:
# import dataset
df = pd.read_csv("./cleaned_data/mental_health_data_2016-2021_ready_for_dashboard.csv")

# drop unnamed columns
df = df.drop('Unnamed: 0', 1)
#df = df.drop('Unnamed: 0.1', 1)
#df = df.drop('age', 1)
#df = df.drop('sex', 1)

In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.


In [7]:
df.head()

,self-employed,have_mental_illness,number_of_employees_your_company_has,sought_mental_treatment,mental_illness_medical_coverage,age_range,sex_0,sex_1,sex_2,working_country_0,...,mental_health_benefits_from_current_employer_3,mental_health_benefits_from_current_employer_4,discussing_mental_health_with_coworkers_0,discussing_mental_health_with_coworkers_1,discussing_mental_health_with_coworkers_2,discussing_mental_health_with_coworkers_3,discussing_mental_health_with_supervisor_0,discussing_mental_health_with_supervisor_1,discussing_mental_health_with_supervisor_2,discussing_mental_health_with_supervisor_3
0,0,0,2,0,0,2,0.0,1.0,0.0,0.0,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,0,1,0,1,0,1,0.0,1.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,0,0,0,1,0,2,0.0,1.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,1,1,5,1,1,2,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
4,0,1,0,1,0,2,1.0,0.0,0.0,0.0,...,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [8]:
# correct column names
arp_data1 = df.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))

In [9]:
# Create train and test datasets
y=arp_data1["have_mental_illness"]
X=arp_data1.drop("have_mental_illness", axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

In [10]:
################################# 1st DASHBOARD -- CATBOOST ####################################
###
###
# Create 1st Explainer Dashboard
model_catb_tuned = catb.CatBoostClassifier(depth=10, iterations=30, learning_rate=0.1)
model_catb_tuned.fit(X_train, y_train)
explainer_catb = ClassifierExplainer(
                model_catb_tuned, X_test, y_test,
                labels=['Had Mental Illness', 'No Mental Ilness']
)

db_catb = ExplainerDashboard(explainer_catb, name='catb', title="CatBoost Explainer",
                    whatif=False,
                    shap_interaction=False,
                    decision_trees=False)

0:	learn: 0.6287238	total: 71.7ms	remaining: 2.08s
1:	learn: 0.5810333	total: 74.3ms	remaining: 1.04s
2:	learn: 0.5418227	total: 80.6ms	remaining: 726ms
3:	learn: 0.5046799	total: 91.6ms	remaining: 595ms
4:	learn: 0.4731639	total: 102ms	remaining: 510ms
5:	learn: 0.4560684	total: 103ms	remaining: 412ms
6:	learn: 0.4365297	total: 113ms	remaining: 372ms
7:	learn: 0.4248256	total: 115ms	remaining: 315ms
8:	learn: 0.4148664	total: 120ms	remaining: 279ms
9:	learn: 0.3982277	total: 129ms	remaining: 259ms
10:	learn: 0.3838708	total: 139ms	remaining: 239ms
11:	learn: 0.3797518	total: 139ms	remaining: 209ms
12:	learn: 0.3651993	total: 148ms	remaining: 194ms
13:	learn: 0.3624916	total: 150ms	remaining: 171ms
14:	learn: 0.3539132	total: 159ms	remaining: 159ms
15:	learn: 0.3423946	total: 167ms	remaining: 146ms
16:	learn: 0.3332793	total: 176ms	remaining: 135ms
17:	learn: 0.3244995	total: 186ms	remaining: 124ms
18:	learn: 0.3151559	total: 195ms	remaining: 113ms
19:	learn: 0.3068229	total: 204ms	rem

In [11]:
################################# 2nd DASHBOARD -- XGBOOST ####################################
###
###
# Create 2nd Explainer Dashboard
import xgboost as xgb

model_xgb_tuned = xgb.XGBClassifier(depth=4, iterations=10, learning_rate=0.02)

model_xgb_tuned.fit(X_train, y_train)

explainer_xgb = ClassifierExplainer(
                model_xgb_tuned, X_test, y_test,
                labels=['Had Mental Illness', 'No Mental Ilness']
)

db_xgb = ExplainerDashboard(explainer_xgb, name='xgb', title="XGBoost Explainer",
                    whatif=False,
                    shap_interaction=False,
                    decision_trees=False)

[13:57:09] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-3.7/xgboost/src/learner.cc:627: 
Parameters: { "depth", "iterations" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


Detected XGBClassifier model: Changing class type to XGBClassifierExplainer...
Note: model_output=='probability'. For XGBClassifier shap values normally get calculated against X_background, but paramater X_background=None, so using X instead
Generating self.shap_explainer = shap.TreeExplainer(model, X, model_output='probability', feature_perturbation='interventional')...
Note: Shap interaction values will not be available. If shap values in probability space are not necessary you can pass model_output='logodds' to get shap values in lo

In [12]:
################################# 3rd DASHBOARD -- LIGHTGBM ####################################
###
###
# Create 3rd Explainer Dashboard

model_lgbm_tuned = ltb.LGBMClassifier(depth=4, iterations=10, learning_rate=0.01)

model_lgbm_tuned.fit(X_train, y_train)

explainer_lgbm = ClassifierExplainer(
                model_lgbm_tuned, X_test, y_test,
                labels=['Had Mental Illness', 'No Mental Ilness']
)

db_lgbm = ExplainerDashboard(explainer_lgbm, name='lgbm', title="LightGBM Explainer",
                    whatif=False,
                    shap_interaction=False,
                    decision_trees=False)

[LightGBM] [Warning] Unknown parameter: depth
[LightGBM] [Warning] Unknown parameter: iterations
Note: model_output=='probability'. For LGBMClassifier shap values normally get calculated against X_background, but paramater X_background=None, so using X instead
Generating self.shap_explainer = shap.TreeExplainer(model, X, model_output='probability', feature_perturbation='interventional')...
Note: Shap interaction values will not be available. If shap values in probability space are not necessary you can pass model_output='logodds' to get shap values in logodds without the need for a background dataset and also working shap interaction values...
Building ExplainerDashboard..
Detected notebook environment, consider setting mode='external', mode='inline' or mode='jupyterlab' to keep the notebook interactive while the dashboard is running...
Generating layout...
Calculating shap values...
Calculating prediction probabilities...
Calculating metrics...
Calculating confusion matrices...
Calcul

In [13]:
# Run all dashboards in a single host
#db_catb, db_lgbm, db_xgb, 
hub = ExplainerHub([db_xgb, db_lgbm, db_catb])
hub.run(8053)

Using random SECRET_KEY: 2ba10644-be4d-4672-8fb1-9f40f94aac1e, please set it on your app.config["SECRET_KEY"]


Building ExplainerDashboard..
Detected notebook environment, consider setting mode='external', mode='inline' or mode='jupyterlab' to keep the notebook interactive while the dashboard is running...
Generating layout...
Calculating dependencies...
Reminder: you can store the explainer (including calculated dependencies) with explainer.dump('explainer.joblib') and reload with e.g. ClassifierExplainer.from_file('explainer.joblib')
Registering callbacks...
Building ExplainerDashboard..
Detected notebook environment, consider setting mode='external', mode='inline' or mode='jupyterlab' to keep the notebook interactive while the dashboard is running...
Generating layout...
Calculating dependencies...
Reminder: you can store the explainer (including calculated dependencies) with explainer.dump('explainer.joblib') and reload with e.g. ClassifierExplainer.from_file('explainer.joblib')
Registering callbacks...
Building ExplainerDashboard..
Detected notebook environment, consider setting mode='exte

 * Running on http://0.0.0.0:8053/ (Press CTRL+C to quit)
127.0.0.1 - - [11/Jun/2022 13:58:09] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [11/Jun/2022 13:58:09] "GET /static/bootstrap.min.css HTTP/1.1" 200 -
127.0.0.1 - - [11/Jun/2022 13:58:09] "GET /static/jquery-3.5.1.slim.min.js HTTP/1.1" 200 -
127.0.0.1 - - [11/Jun/2022 13:58:09] "GET /static/bootstrap.min.js HTTP/1.1" 200 -
127.0.0.1 - - [11/Jun/2022 13:58:09] "GET /index/ HTTP/1.1" 200 -
127.0.0.1 - - [11/Jun/2022 13:58:09] "GET /index/_dash-component-suites/dash/deps/polyfill@7.v2_3_1m1651157205.12.1.min.js HTTP/1.1" 200 -
127.0.0.1 - - [11/Jun/2022 13:58:09] "GET /index/assets/bootstrap.min.css?m=1651904588.9278898 HTTP/1.1" 200 -
127.0.0.1 - - [11/Jun/2022 13:58:09] "GET /index/_dash-component-suites/dash/deps/react@16.v2_3_1m1651157205.14.0.min.js HTTP/1.1" 200 -
127.0.0.1 - - [11/Jun/2022 13:58:09] "GET /index/_dash-component-suites/dash/deps/react-dom@16.v2_3_1m1651157205.14.0.min.js HTTP/1.1" 200 -
127.0.0.1 - - [11/Jun/2022 13: